# Análisis morfosintáctico y reconomiento de entidades

Se trata de algunas de las tareas más básicas que se realizan a la hora de procesar lenguaje natural:

- [análisis morfológico](https://en.wikipedia.org/wiki/Part-of-speech_tagging) es la tarea que consiste en etiquetar cada palabra de una oración de acuerdo con su categoría y sus rasgos gramaticales.   

![pos tagging](https://drive.google.com/uc?id=1iczT9qpy68IFxuEtML-ICuV_YU2ttZit)

- [análisis sintáctico](https://en.wikipedia.org/wiki/Parsing) es la tarea que consiste analizar la estructura de un texto para construir el árbol sintáctico de e identificar  indicando las funciones que cumplen cada uno de los constituyentes o elementos de la oración.

![parsig: dependencies](https://drive.google.com/uc?id=1svJNxpFmC1HSUSUX5x-_cqFtfTJEtwcX)

- [reconocimiento de entidades nombradas](https://en.wikipedia.org/wiki/Named-entity_recognition) es la tarea que consiste en identificar qué palabras de un texto hacen referencia a entidades del mundo y qué tipo de entidad es en cada caso. Estos tipos de entidades pueden ser *generalistas* (nombres propios que hacen referencia a personas, organizaciones, lugares, productos, fechas, cantidades, etc) o pueden ser *específicas* para un dominio concreto (retail: marcas y modelos de productos, colores, tallas, tejidos; bio médico: ARN, proteínas, tipos de células, ADN; banca: direcciones, DNI, referencia catastral, importes, tipo de cargas, nombres de notario, nombre de titular). 

En ambos casos, estamos asumiendo varias cosas:

1. que contamos con lenguaje natural en formato texto, y;
2. que somos capaces de segmentar correctamente el texto en oraciones y palabras.

Veamos antes un poco más en profundidad qué es la tokenización y cómo afecta este paso al resto de tareas.

## Tokenización

Tokenización es la tarea que consiste en segmentar un texto en *tokens* o unidades mínimas. En muchos casos estos tokens coinciden con nuestra idea intuitiva de *palabra*, pero no tiene por qué. Es más una decisión de diseño. 

Por cierto, ¿qué es una palabra?

![words](https://drive.google.com/uc?id=1VhASH6n-mS4SmWtDkrpn9DQ46O9cmUpr)

In [ ]:
texto = """«Imagine» es una canción interpretada por el músico inglés John Lennon y escrita por Yoko Ono y él a principios de 1971. Si bien el sello discográfico Apple Records distribuyó la canción a partir de octubre de ese año en Estados Unidos, esta no estuvo disponible en Reino Unido hasta 1975. Se trata de una balada de piano del género soft rock cuya letra hace alusión a una variedad de conceptos, aunque primordialmente promueve la idea de que «todos somos una nación, un mundo y una sociedad». En 1987 se estrenó un videoclip para promocionar la canción, bajo la dirección de Zbigniew Rybczyński."""

In [ ]:
tokens = texto.split()

print(f"Rompiendo la cadena son split, tenemos {len(tokens)} tokens.\n")
print(tokens)

In [ ]:
import nltk

nltk_tokens = nltk.word_tokenize(texto)

print(f"Rompiendo la cadena son NLTK, tenemos {len(nltk_tokens)} tokens.\n")
print(nltk_tokens)

In [ ]:
# Si esta celda falla, prueba a instalar antes el corpus cess_esp
# nltk.download("cess_esp")

from nltk.corpus import cess_esp

# cargo el primer documento del corpus segmentado en palabras
palabras = cess_esp.words(cess_esp.fileids()[0])[:99]
print(palabras)

¿Y qué hacemos cuando el texto a tokenizar no solo contiene palabras. 

In [ ]:
tweet = """Have more than one Twitter account? You can toggle between them on your app! 📱 Just follow these handy #TwitterTips: https://t.co/MeqwfQQqah?amp=1 😀"""

from nltk.tokenize import TweetTokenizer

t = TweetTokenizer()

tweet_tokens = t.tokenize(tweet)
print(f"Usando el TweetTokenizer de NLTK ahora tenemos {len(tweet_tokens)} tokens.\n")
print(tweet_tokens)

# `spaCy`

## Análisis Morfosintáctico con y Reconomiento de entidades con `spaCy`

[`spaCy`](http://www.spacy.io/) es una librería de procesamiento del lenguaje natural, robusta, rápida, fácil de instalar y utilizar e integrable con [otras librerías de *NLP* y de *deep learning*](https://spacy.io/usage/facts-figures#section-other-libraries). 

Tiene unos cuantos [modelos ya entrenados en varios idiomas](https://spacy.io/usage/models) y permite realizar las [típicas tareas](https://spacy.io/usage/facts-figures) de segmentación por oraciones, tokenizanción, análisis morfológico, extracción de entidades y análisis de opinión.

Empezamos cargando un modelo en español.

In [ ]:
import spacy

# cargamos el modelo entrenado en español
nlp = spacy.load("es_core_news_md")

# si la carga de modelos no funciona, prueba a instalarlos antes
#!python -m spacy download es_core_news_md
#!python -m spacy download en_core_web_md

In [ ]:
# analizamos el texto de entrada
doc = nlp(texto)

# y podemos iterar sobre las oraciones de doc
for n, oracion in enumerate(doc.sents):
    print(f"\nOración {n+1}: {oracion}\n")
    
    # para imprimir la información morfo-sintáctica de cada token
    for token in oracion:
        print(f"{token.text} tiene lema {token.lemma_}, categoría {token.tag_} y función {token.dep_}")

A veces es útil poder visualizar el árbol de dependencias. Podemos hacerlo directamente desde aquí.

In [ ]:
from spacy import displacy

displacy.serve(doc, style="dep")

El objeto `doc` que hemos creado al analizar el texto de entrada también ya ejecutado el reconocimiento de entidades y es capaz de reconocer hasta [4 tipos de entidades: PER, LOC, ORG, MISC](https://spacy.io/api/annotation#ner-wikipedia-scheme).

In [ ]:
print(f"El texto tiene {len(doc.ents)} entidades distintas.\n")

for entidad in doc.ents:
    print(f"La entidad {entidad} es del tipo {entidad.label_}.")

Al igual que con el análisis de dependencias, podemos visualizar las entidades nombradas de manera gráfica.

In [ ]:
from spacy import displacy

displacy.serve(doc, style="ent")

¿Qué tal funciona en inglés? Carguemos un modelo probemos con algunos textos.

In [ ]:
nlp_en = spacy.load("en_core_web_md")

text = """After Apple discovered in June that certain MacBook laptops could overheat, posing a fire hazard, the Consumer Product Safety Commission quickly issued a warning, along with information about consumer burns and smoke inhalation. But after Apple learned that its FaceTime video chat app was enabling consumers to listen in on the conversations of people they called — even when the recipients did not answer their phones — there was no designated federal protection agency to warn Americans or collect reports of privacy invasions."""

doc_en = nlp_en(text)

In [ ]:
# y podemos iterar sobre las oraciones de doc
for n, oracion in enumerate(doc_en.sents):
    print(f"\nOración {n+1}: {oracion}\n")
    
    # para imprimir la información morfo-sintáctica de cada token
    for token in oracion:
        print(f"{token.text} tiene lema {token.lemma_}, categoría {token.tag_} y función {token.dep_}")

Depende del tipo de textos con los que problemos, veremos mejores o peores resultados. Debemos tener siempre en cuenta el tipo de lengua con el que modelo ha sido entrenado. Y los nombres de los modelos de `spaCy` nos dan una pista: 

- [`es_core_news_md`](https://spacy.io/models/es#es_core_news_md) es el modelo mediano entrenado en español con datos de noticias (lengua estándar, variante culta).

- [`en_core_web_md`](https://spacy.io/models/en#en_core_web_md) es el modelo mediano entrenado en inglés con datos de la web (una mezcla de textos de Wikipedia y medios online, blogs, comentarios de redes sociales y foros, con muestras de lengua culta pero también usos coloquiales).

Lo que sí funciona menor en inglés es el reconocimiento de entidades, porque [los datos de entrenamiento contienen más tipos de entidad](https://spacy.io/api/annotation#named-entities). 

In [ ]:
print(f"El texto tiene {len(doc_en.ents)} entidades distintas.\n")

for entidad in doc_en.ents:
    print(f"La entidad {entidad} es del tipo {entidad.label_}.")

# `flair`

La compañia Zalando tiene necesidades de aplicar NLP en distintos ámbitos y su equipo de investigación ha liberado recientemente [`flair`](https://github.com/zalandoresearch/flair), su librería de NLP.

`flair` permite acceder a funcionalidades muy interesantes para procesar lenguaje natural, algunas de ellas muy modernas como:

- [etiquetar morfo-sintácticamente](https://github.com/zalandoresearch/flair/blob/master/resources/docs/TUTORIAL_2_TAGGING.md)
- extraer entidades
- clasificar automáticamente texto
- entrenar tus propios modelos para [construir otros clasificadores](https://towardsdatascience.com/text-classification-with-state-of-the-art-nlp-library-flair-b541d7add21f)
- [cargar vectores de palabras en decenas de lenguas](https://github.com/zalandoresearch/flair/blob/master/resources/docs/TUTORIAL_3_WORD_EMBEDDING.md)
- [usar vectores contextuales como BERT, ELMo](https://github.com/zalandoresearch/flair/blob/master/resources/docs/TUTORIAL_4_ELMO_BERT_FLAIR_EMBEDDING.md)

## Análisis Morfosintáctico con y Reconomiento de entidades con `flair`

Para analizar sintácticamente un texto, necesitamos cargar un [etiquetador con un modelo concreto de información morfo-sintáctica](https://github.com/zalandoresearch/flair/blob/master/resources/docs/TUTORIAL_2_TAGGING.md). Por ejemplo, uno en inglés.

In [ ]:
from flair.data import Sentence
from flair.models import SequenceTagger

# cargamos el analizador multi-idioma
tagger = SequenceTagger.load("pos")

In [ ]:
sentence = Sentence("Why are Americans protected from hazardous laptops, fitness trackers and smartphones — but not when hazardous apps on our devices expose and exploit our personal information?")

tagger.predict(sentence)

print(sentence.to_tagged_string())

`flair` tiene una cosa muy interesante, un modelo con información morfológica multi-idioma. Veamos como funciona:

In [ ]:
from flair.data import Sentence
from flair.models import SequenceTagger

# cargamos el analizador multi-idioma
multi_tagger = SequenceTagger.load("pos-multi-fast")

In [ ]:
sentence1 = Sentence("Facebook nació hace década y media tras una noche de copas de Mark Zuckerberg. ")
multi_tagger.predict(sentence1)
# imprimimos el análisis
print(sentence1.to_tagged_string(), "\n")

sentence2 = Sentence("Grand débat national: suivez Emmanuel Macron en direct de Bordeaux. ")
multi_tagger.predict(sentence2)
# imprimimos el análisis
print(sentence2.to_tagged_string(), "\n")

sentence3 = Sentence("Hier an der Zufahrt zur Startrampe 39A, wo vor 50 Jahren die gigantischen Saturn-Raketen der Apollo-Mondmissionen im Schneckentempo vorbeigefahren sind, prangen nun die blauen Lettern des Raumfahrtunternehmens von Elon Musk an einem Hangar.")
multi_tagger.predict(sentence3)
# imprimimos el análisis
print(sentence3.to_tagged_string(), "\n")

Para probar el reconocimiento de entidades, solo necesitamos cargar un modelo preentrenado con información de este tipo. Por ejemplo:

In [ ]:
from flair.data import Sentence
from flair.models import SequenceTagger

# cargamos el reconocedor de entidades
multi_ner = SequenceTagger.load("ner-multi-fast")

multi_ner.predict(sentence1)

In [ ]:
# imprimimos el análisis
print(sentence1.to_tagged_string())

# iteramos por la entidades
for entity in sentence1.get_spans("ner"):
    print(entity)
    
# o imprimimos la estructura de datos con el análisis completo
print(sentence1.to_dict(tag_type="ner"))

Para el inglés, existe un modelo entrenado con el mismo corpus que `spaCy`, aunque según los autores de `flair` ellos consiguen mejores resultados. Veamos:

In [ ]:
from flair.data import Sentence
from flair.models import SequenceTagger

# cargamos el reconocedor de entidades
ner = SequenceTagger.load("ner-ontonotes-fast")

In [ ]:
sentence4 = Sentence("Behind closed doors, freshman Rep. Alexandria Ocasio-Cortez threatened to put those voting with Republicans “on a list” for a primary challenge in the 2020 election.")
ner.predict(sentence4)

# imprimimos el análisis
print(sentence4.to_tagged_string())

# iteramos por la entidades
for entity in sentence4.get_spans("ner"):
    print(entity)